# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합 - pipeline**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * 단위 테스트
        * 각 기능(함수)에 대해 단계별로 테스트를 수행하며 오류를 해결합니다.
    * 파이프라인 구축
        * 단계1의 결과가 단계2 모델에 input이 되고, 모델의 예측 결과를 기반으로
        * 응급실 추천되도록
        * 조원들이 녹음한 음성 파일에 임의의 좌표(위도, 경도)값을 부여
            * 음성파일 이름과 좌표를 저장하는 별도 데이터셋 생성
        * 각 모듈을 연결하여 파이프라인 구성하는 ipynb 파일 생성



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [10]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [11]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 조에서 생성한 모듈 불러오기 -------------
import emergency as em

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [ ]:
  # GPT-3.5, API_KEY 불러오기
  with open(path + 'api_key.txt', 'r') as file:
    openai.api_key = file.readline().strip()

In [ ]:
em.register_key(openai.api_key)

### (2) audio to text

In [ ]:
text = em.audio_to_text(path+'audio/', 'audio1.mp3')

In [ ]:
print(text)

지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.



### (3) text summary

In [ ]:
summary = em.text_summary(text)

In [ ]:
print(summary)

{"summary": "급히 응급실로 이동하는 것이 좋을 것입니다. 넘어진 후 머리에서 피가 나오고 어지러움을 호소하고 있는 경우, 심각한 상황일 수 있으므로 응급 의료가 필요합니다. 물을 마시게 하며 응급실로 이동하는 것이 가장 안전한 조치일 것입니다."}


### (4) 응급실 등급분류

In [30]:
# BERT Model 불러오기
save_dir = path + 'fine_tuned_bert'

# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(save_dir)
tokenizer = AutoTokenizer.from_pretrained(save_dir)

In [32]:
emer_lev, prob = em.predict(summary, model, tokenizer)

In [33]:
print(emer_lev, prob)

3 tensor([[0.0175, 0.0275, 0.5027, 0.2688, 0.1835]])


### (5) 응급실추천

In [7]:
# Directions5 API_ID, API_KEY 불러오기
with open(path + 'map_key.txt', 'r') as file:
  data = json.load(file)
c_id = data['c_id']
c_key = data['c_key']

# 응급실 데이터프레임 불러오기
emergency_df = pd.read_csv(path + 'df_hos.csv')

In [8]:
# KT분당본사타워 위도, 경도, 응급실_데이터프레임
dist_list = em.emergency_recommendation(37.359, 127.114,
                                        emergency_df, c_id, c_key)

In [9]:
dist_list

[('분당서울대학교병원',
  '031-787-2114',
  '경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)',
  2.828),
 ('대진의료재단분당제생병원', '031-779-0114', '경기도 성남시 분당구 서현로180번길 20 (서현동)', 4.909),
 ('국군수도병원', '031-725-6075', '경기도 성남시 분당구 새마을로177번길 81 (율동)', 6.175)]

## **3. 파이프라인**

* 세부사항
    * [2. 단계별 테스트] 의 내용을 순차적으로 정리합니다.
        * 데이터 처리 전 준비작업 : 한번 실행하면 되는 영역
            * 키, 데이터로딩
            * 모델/토크나이저 로딩
        * 입력값이 들어 왔을 때 출력값까지 처리되는 영역

In [12]:
# audio : 'audio.mp3'
# la, lo : float, float
def pipeline(audio, la, lo) :
  print("0 / 8...")
  # 기본 데이터 로딩
    # 0. Path 지정
  path = '/content/drive/MyDrive/project6_2/'

    # 1. GPT-3.5, API_KEY 불러오기
  with open(path + 'api_key.txt', 'r') as file:
    openai.api_key = file.readline().strip()

    # 2. Directions5 API_ID, API_KEY 불러오기
  with open(path + 'map_key.txt', 'r') as file:
    data = json.load(file)
  c_id = data['c_id']
  c_key = data['c_key']

    # 3. 응급실 데이터프레임 불러오기
  emergency_df = pd.read_csv(path + 'df_hos.csv')

    # 4. BERT Model 불러오기
  save_dir = path + 'fine_tuned_bert'

  print("1 / 8... \t|\t모델 불러오는 중")
  # 모델 로드
  model = AutoModelForSequenceClassification.from_pretrained(save_dir)
  tokenizer = AutoTokenizer.from_pretrained(save_dir)

  print("2 / 8... \t|\tOPEN AI KEY 등록 중")
  # open ai key 등록
  em.register_key(openai.api_key)

  print("3 / 8... \t|\t음성을 문장으로 변환 중")
  # Audio to Text
  text = em.audio_to_text(path+'audio/', audio)

  print("4 / 8... \t|\t문장 요약 중")
  # Text to Summary
  summary = em.text_summary(text)

  print("5 / 8... \t|\t등급 분류 중")
  # 응급실 등급 분류
  emer_lev, prob = em.predict(summary, model, tokenizer)

  print("6 / 8... \t|\t등급 확인 중")

  # (OPT) 등급에 따른 응급실 호출 여부 로직 구현
  if emer_lev == 4 :
    if prob[0][0] + prob[0][1] + prob[0][2] > prob[0][3] +prob[0][4] :
      print(f'KTAS 등급:{emer_lev}\t|\t응급상황입니다. 가까운 응급실 3곳을 선별합니다.')
    else :
      print(f'KTAS 등급:{emer_lev}\t|\t응급상황이 아닙니다.')
      return []
  else :
    if emer_lev < 4 :
      print(f'KTAS 등급:{emer_lev}\t|\t응급상황입니다. 가까운 응급실 3곳을 선별합니다.')
    else :
      print(f'KTAS 등급:{emer_lev}\t|\t응급상황이 아닙니다.')
      return []

  print("7 / 8... \t|\t응급실 선별 중")

  # 가까운 응급실 3곳 추천
  dist_list = em.emergency_recommendation(la, lo, emergency_df, c_id, c_key)

  print("8 / 8... \t|\t완료")
  return dist_list

In [13]:
emer_list = pipeline('audio1.mp3', 37.359775085276, 127.11468651854)

0 / 8...
1 / 8... 	|	모델 불러오는 중
2 / 8... 	|	OPEN AI KEY 등록 중
3 / 8... 	|	음성을 문장으로 변환 중
4 / 8... 	|	문장 요약 중
5 / 8... 	|	등급 분류 중
6 / 8... 	|	등급 확인 중
KTAS 등급:3	|	응급상황입니다. 가까운 응급실 3곳을 선별합니다.
7 / 8... 	|	응급실 선별 중
8 / 8... 	|	완료


In [14]:
import unicodedata

def get_string_width(s):
    """Calculate the display width of a string considering full-width characters."""
    return sum(2 if unicodedata.east_asian_width(c) in "WF" else 1 for c in s)

# Calculate maximum widths for each column
max_name_width = max(get_string_width(hospital) for hospital, _, _, _ in emer_list)
max_phone_width = max(get_string_width(phone) for _, phone, _, _ in emer_list)
max_address_width = max(get_string_width(address) for _, _, address, _ in emer_list)

# Header
print(f"{'병원 이름':<{max_name_width}} {'전화번호':<{max_phone_width}} {'주소':<{max_address_width}} {'거리(KM)':<5}")
print("-" * (max_name_width + max_phone_width + max_address_width + 10))

# Rows
for hospital, phone, address, score in emer_list:
    # Calculate padding based on string width
    name_padding = " " * (max_name_width - get_string_width(hospital))
    phone_padding = " " * (max_phone_width - get_string_width(phone))
    address_padding = " " * (max_address_width - get_string_width(address))

    print(f"{hospital}{name_padding} {phone}{phone_padding} {address}{address_padding} {score:<5.3f}")

print("-" * (max_name_width + max_phone_width + max_address_width + 10))

병원 이름                    전화번호         주소                                                                 거리(KM)
----------------------------------------------------------------------------------------------------------------
분당서울대학교병원       031-787-2114 경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원) 2.400
대진의료재단분당제생병원 031-779-0114 경기도 성남시 분당구 서현로180번길 20 (서현동)                     4.481
국군수도병원             031-725-6075 경기도 성남시 분당구 새마을로177번길 81 (율동)                     5.747
----------------------------------------------------------------------------------------------------------------
